<a href="https://colab.research.google.com/github/moey920/Object-Detection/blob/master/%EC%A0%95%EC%88%98_%EC%9D%B8%EC%BD%94%EB%94%A9(Integer_Encoding).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정수 인코딩(Integer Encoding)

컴퓨터는 텍스트보다는 숫자를 더 잘 처리 할 수 있습니다. 이를 위해 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있습니다. 그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있습니다.

예를 들어 갖고 있는 텍스트에 단어가 5,000개가 있다면, 5,000개의 단어들 각각에 1번부터 5,000번까지 단어와 맵핑되는 고유한 정수, 다른 표현으로는 인덱스를 부여합니다. 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 숫자가 부여됩니다. 인덱스를 부여하는 방법은 여러 가지가 있을 수 있는데 랜덤으로 부여하기도 하지만, 보통은 전처리 또는 빈도수가 높은 단어들만 사용하기 위해서 단어에 대한 빈도수를 기준으로 정렬한 뒤에 부여합니다.

## 정수 인코딩(Integer Encoding) 실습
단어에 정수를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary)을 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법이 있습니다. 이해를 돕기위해 단어의 빈도수가 적당하게 분포되도록 의도적으로 만든 텍스트 데이터를 가지고 실습해보겠습니다.

###  1) dictionary 사용하기

In [0]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
f = open('/content/drive/My Drive/text/data.txt', 'rt', encoding='UTF8')
text = f.read()
text

'올리브\n영\n이정\n부대\n로\n경기도\n의정부시\n평화\n로\n552\n031\n856\n5290\n1274446611\n장\n유진\nTEL\n장\n유진\n2020\n-02\n-24\n18\n40\n33\n1001\n00077\n고객\n센터\n1577\n4887\noliveyoung\n결제\n일로\n부터\n15\n일\n3\n월\n10\n일\n판매\n가능\n하\nㄴ\n정\n상품\n미\n개봉\n결제\n하\n었\n더\nㄴ\n수단\n구매\n영수증\n매장\n방문\n가능\n결제\n수\n변경\n구\n매매장\n에서\n가능\n부분\n영수증\n선\n증\n의하\nㄴ\n환\n불은\n불가\n하\n며\n부분\n영수증\n선불\n증\n매장\n방문\n해당\n금액\n이상\n상품\n기능\n증정\n품\n미\n불가\n클리어런스\n상품\n불가\n재\n발행\n영수증\n카드\n전표\n고객\n용\n상품명\n단가\n수량\n금액\n10\n쏘피\n오가\n크트\n라이더\n일반\n매가\n행사\n8809299236129\n5\n,\n400\n2\n5\n,\n400\n상\n품\n표시\n품목\n부\n가세\n면세\n품목\n클리어런스\n쿠폰\n제휴\n,\n중복\n불가\nOLI\n944\n||\nYOU\n4,860\n판매계\n5\n,\n400\n신용카드\n14\n,\n860\n제휴\n금액\n540\n계제\n그\n4,660\n2\n세대\n1\n승인\n금액\n4,860\n카\n아\n5433\n334\n#\n#\n카드\n사\n명\n현대\n카드\n승인\n번호\n00146478\n일시불\n심\n영\n님\nCJ\nONE\n포인트\n내역\n회원\n번호\n**\n***\n023635\n적립\n포인트\n1\n24\n가용\n포인트\n15,319\n적립\n포인트\n는\n익일\n가용\n포인트\n로\n전환\n자세\n하\nㄴ\n포인트\n내역\n1\n천\n포인트\n이상\n사용\n가능\nOLIVE\nYOUNG\n12020022411000506100100077\nCJ\n올리브\n영\n개\n롱\n역점\n서울특별시\n송파구\n오금\n로\n412\n,\

In [5]:
# 문장 토큰화
import nltk
nltk.download('punkt')
text=sent_tokenize(text)
print(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['올리브\n영\n이정\n부대\n로\n경기도\n의정부시\n평화\n로\n552\n031\n856\n5290\n1274446611\n장\n유진\nTEL\n장\n유진\n2020\n-02\n-24\n18\n40\n33\n1001\n00077\n고객\n센터\n1577\n4887\noliveyoung\n결제\n일로\n부터\n15\n일\n3\n월\n10\n일\n판매\n가능\n하\nㄴ\n정\n상품\n미\n개봉\n결제\n하\n었\n더\nㄴ\n수단\n구매\n영수증\n매장\n방문\n가능\n결제\n수\n변경\n구\n매매장\n에서\n가능\n부분\n영수증\n선\n증\n의하\nㄴ\n환\n불은\n불가\n하\n며\n부분\n영수증\n선불\n증\n매장\n방문\n해당\n금액\n이상\n상품\n기능\n증정\n품\n미\n불가\n클리어런스\n상품\n불가\n재\n발행\n영수증\n카드\n전표\n고객\n용\n상품명\n단가\n수량\n금액\n10\n쏘피\n오가\n크트\n라이더\n일반\n매가\n행사\n8809299236129\n5\n,\n400\n2\n5\n,\n400\n상\n품\n표시\n품목\n부\n가세\n면세\n품목\n클리어런스\n쿠폰\n제휴\n,\n중복\n불가\nOLI\n944\n||\nYOU\n4,860\n판매계\n5\n,\n400\n신용카드\n14\n,\n860\n제휴\n금액\n540\n계제\n그\n4,660\n2\n세대\n1\n승인\n금액\n4,860\n카\n아\n5433\n334\n#\n#\n카드\n사\n명\n현대\n카드\n승인\n번호\n00146478\n일시불\n심\n영\n님\nCJ\nONE\n포인트\n내역\n회원\n번호\n**\n***\n023635\n적립\n포인트\n1\n24\n가용\n포인트\n15,319\n적립\n포인트\n는\n익일\n가용\n포인트\n로\n전환\n자세\n하\nㄴ\n포인트\n내역\n1\n천\n

기존의 텍스트 데이터가 문장 단위로 토큰화 된 것을 확인할 수 있습니다. 이제 정제 작업을 병행하며, 단어 토큰화를 수행합니다.

In [28]:
# 정제와 단어 토큰화
nltk.download('stopwords')

vocab={} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence=word_tokenize(i) # 단어 토큰화를 수행합니다.
    result = []

    for word in sentence: 
        word=word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
            if len(word) > 1: # 단어 길이가 1이하인 경우에 대하여 추가로 단어를 제거합니다.
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0 
                vocab[word] += 1
    sentences.append(result) 
print(sentences)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[['올리브', '이정', '부대', '경기도', '의정부시', '평화', '552', '031', '856', '5290', '1274446611', '유진', 'tel', '유진', '2020', '-02', '-24', '18', '40', '33', '1001', '00077', '고객', '센터', '1577', '4887', 'oliveyoung', '결제', '일로', '부터', '15', '10', '판매', '가능', '상품', '개봉', '결제', '수단', '구매', '영수증', '매장', '방문', '가능', '결제', '변경', '매매장', '에서', '가능', '부분', '영수증', '의하', '불은', '불가', '부분', '영수증', '선불', '매장', '방문', '해당', '금액', '이상', '상품', '기능', '증정', '불가', '클리어런스', '상품', '불가', '발행', '영수증', '카드', '전표', '고객', '상품명', '단가', '수량', '금액', '10', '쏘피', '오가', '크트', '라이더', '일반', '매가', '행사', '8809299236129', '400', '400', '표시', '품목', '가세', '면세', '품목', '클리어런스', '쿠폰', '제휴', '중복', '불가', 'oli', '944', '||', '4,860', '판매계', '400', '신용카드', '14', '860', '제휴', '금액', '540', '계제', '4,660', '세대', '승인', '금액', '4,860', '5433', '334', '카드', '현대', '카드', '승인', '번호', '00146478', '일시불', 'cj', 'one', '포인트', '내역', '회원', '번호',

텍스트를 숫자로 바꾸는 단계라는 것은 본격적으로 자연어 처리 작업에 들어간다는 의미이므로, 단어가 텍스트일 때만 할 수 있는 최대한의 전처리를 끝내놓아야 합니다. 위의 코드를 보면, 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트되는 일이 없도록 모든 단어를 소문자로 바꾸었습니다. 그리고 자연어 처리에서 크게 의미를 갖지 못하는 불용어와 길이가 짧은 단어를 제거하는 방법을 사용하였습니다.

현재 vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록되어져 있습니다. vocab을 출력해보겠습니다.

In [29]:
print(vocab)

{'올리브': 232, '이정': 2, '부대': 1, '경기도': 35, '의정부시': 2, '평화': 1, '552': 1, '031': 33, '856': 1, '5290': 40, '1274446611': 1, '유진': 4, 'tel': 210, '2020': 164, '-02': 44, '-24': 4, '18': 111, '40': 21, '33': 12, '1001': 84, '00077': 1, '고객': 358, '센터': 189, '1577': 182, '4887': 140, 'oliveyoung': 165, '결제': 715, '일로': 178, '부터': 194, '15': 277, '10': 189, '판매': 314, '가능': 886, '상품': 611, '개봉': 194, '수단': 372, '구매': 223, '영수증': 762, '매장': 469, '방문': 402, '변경': 199, '매매장': 162, '에서': 195, '부분': 380, '의하': 192, '불은': 160, '불가': 737, '선불': 2, '해당': 201, '금액': 689, '이상': 387, '기능': 26, '증정': 194, '클리어런스': 386, '발행': 38, '카드': 618, '전표': 160, '상품명': 197, '단가': 197, '수량': 200, '쏘피': 4, '오가': 1, '크트': 1, '라이더': 1, '일반': 1, '매가': 233, '행사': 241, '8809299236129': 1, '400': 76, '표시': 203, '품목': 397, '가세': 15, '면세': 202, '쿠폰': 296, '제휴': 219, '중복': 207, 'oli': 10, '944': 1, '||': 6, '4,860': 2, '판매계': 38, '신용카드': 148, '14': 75, '860': 1, '540': 2, '계제': 3, '4,660': 1, '세대': 1, '승인': 344, '5433': 1, '3

단어를 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어져 있습니다. vocab에 단어를 입력하면 빈도수를 리턴합니다.

In [30]:
print(vocab["young"])

70


이제 빈도수가 높은 순서대로 정렬해보겠습니다.

In [31]:
vocab_sorted=sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

[('포인트', 1256), ('가능', 886), ('영수증', 762), ('불가', 737), ('결제', 715), ('금액', 689), ('카드', 618), ('상품', 611), ('번호', 471), ('매장', 469), ('000', 416), ('방문', 402), ('품목', 397), ('이상', 387), ('클리어런스', 386), ('부분', 380), ('수단', 372), ('선물', 371), ('내역', 364), ('고객', 358), ('cj', 357), ('적립', 355), ('가용', 355), ('승인', 344), ('**', 317), ('판매', 314), ('쿠폰', 296), ('15', 277), ('행사', 241), ('매가', 233), ('올리브', 232), ('구매', 223), ('제휴', 219), ('합계', 216), ('tel', 210), ('중복', 207), ('02', 206), ('표시', 203), ('면세', 202), ('해당', 201), ('수량', 200), ('변경', 199), ('상품명', 197), ('단가', 197), ('에서', 195), ('부터', 194), ('개봉', 194), ('증정', 194), ('의하', 192), ('센터', 189), ('10', 189), ('과세', 187), ('one', 185), ('1577', 182), ('사용', 182), ('익일', 181), ('일로', 178), ('자세', 177), ('일시불', 171), ('oliveyoung', 165), ('2020', 164), ('전환', 164), ('매매장', 162), ('12', 162), ('불은', 160), ('전표', 160), ('회원', 159), ('900', 157), ('11', 151), ('신용카드', 148), ('4887', 140), ('500', 134), ('8098101574', 125), ('18', 111)

이제 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여합니다.

In [32]:
word_to_index={}
i=0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 정제(Cleaning) 챕터에서 언급했듯이 빈도수가 적은 단어는 제외한다.
        i=i+1
        word_to_index[word]=i
print(word_to_index)

{'포인트': 1, '가능': 2, '영수증': 3, '불가': 4, '결제': 5, '금액': 6, '카드': 7, '상품': 8, '번호': 9, '매장': 10, '000': 11, '방문': 12, '품목': 13, '이상': 14, '클리어런스': 15, '부분': 16, '수단': 17, '선물': 18, '내역': 19, '고객': 20, 'cj': 21, '적립': 22, '가용': 23, '승인': 24, '**': 25, '판매': 26, '쿠폰': 27, '15': 28, '행사': 29, '매가': 30, '올리브': 31, '구매': 32, '제휴': 33, '합계': 34, 'tel': 35, '중복': 36, '02': 37, '표시': 38, '면세': 39, '해당': 40, '수량': 41, '변경': 42, '상품명': 43, '단가': 44, '에서': 45, '부터': 46, '개봉': 47, '증정': 48, '의하': 49, '센터': 50, '10': 51, '과세': 52, 'one': 53, '1577': 54, '사용': 55, '익일': 56, '일로': 57, '자세': 58, '일시불': 59, 'oliveyoung': 60, '2020': 61, '전환': 62, '매매장': 63, '12': 64, '불은': 65, '전표': 66, '회원': 67, '900': 68, '11': 69, '신용카드': 70, '4887': 71, '500': 72, '8098101574': 73, '18': 74, '100': 75, '구창': 76, '13': 77, '800': 78, '1001': 79, '19': 80, '****': 81, '20': 82, '1002': 83, '26': 84, '400': 85, '14': 86, '부가세': 87, '200': 88, 'young': 89, 'olive': 90, '사명': 91, '서울특별시': 92, '***': 93, '300': 94, '16': 95

1의 인덱스를 가진 단어가 가장 빈도수가 높은 단어가 됩니다. 그리고 이러한 작업을 수행하는 동시에 각 단어의 빈도수를 알 경우에만 할 수 있는 전처리인 빈도수가 적은 단어를 제외시키는 작업을 합니다. 등장 빈도가 낮은 단어는 자연어 처리에서 의미를 가지지 않을 가능성이 높기 때문입니다. 여기서는 빈도수가 1인 단어들은 전부 제외시켰습니다.

자연어 처리를 하다보면, 텍스트 데이터에 있는 단어를 모두 사용하기 보다는 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많습니다. 위 단어들은 빈도수가 높은 순으로 낮은 정수가 부여되어져 있으므로 빈도수 상위 n개의 단어만 사용하고 싶다고하면 vocab에서 정수값이 1부터 n까지인 단어들만 사용하면 됩니다. 여기서는 상위 50개 단어만 사용한다고 가정하겠습니다.

In [33]:
vocab_size=50
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
print(word_to_index)

{'포인트': 1, '가능': 2, '영수증': 3, '불가': 4, '결제': 5, '금액': 6, '카드': 7, '상품': 8, '번호': 9, '매장': 10, '000': 11, '방문': 12, '품목': 13, '이상': 14, '클리어런스': 15, '부분': 16, '수단': 17, '선물': 18, '내역': 19, '고객': 20, 'cj': 21, '적립': 22, '가용': 23, '승인': 24, '**': 25, '판매': 26, '쿠폰': 27, '15': 28, '행사': 29, '매가': 30, '올리브': 31, '구매': 32, '제휴': 33, '합계': 34, 'tel': 35, '중복': 36, '02': 37, '표시': 38, '면세': 39, '해당': 40, '수량': 41, '변경': 42, '상품명': 43, '단가': 44, '에서': 45, '부터': 46, '개봉': 47, '증정': 48, '의하': 49, '센터': 50}


이제 word_to_index에는 빈도수가 높은 상위 50개의 단어만 저장되었습니다. 이제 word_to_index를 사용하여 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾸는 작업을 하겠습니다.

예를 들어 sentences에서 첫번째 문장은 ['barber', 'person']이었는데, 이 문장에 대해서는 [1, 5]로 인코딩합니다. 그런데 두번째 문장인 ['barber', 'good', 'person']에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어가 있습니다.

이처럼 훈련 데이터 또는 테스트 데이터에 대해서 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 'OOV'라고 합니다. word_to_index에 'OOV'란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩하겠습니다.

In [0]:
word_to_index['OOV']=len(word_to_index)+1

이제 word_to_index를 사용하여 sentences의 모든 단어들을 맵핑되는 정수로 인코딩하겠습니다.

In [35]:
encoded=[]
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[31, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 35, 51, 51, 51, 51, 51, 51, 51, 51, 51, 20, 50, 51, 51, 51, 5, 51, 46, 28, 51, 26, 2, 8, 47, 5, 17, 32, 3, 10, 12, 2, 5, 42, 51, 45, 2, 16, 3, 49, 51, 4, 16, 3, 51, 10, 12, 40, 6, 14, 8, 51, 48, 4, 15, 8, 4, 51, 3, 7, 51, 20, 43, 44, 41, 6, 51, 51, 51, 51, 51, 51, 30, 29, 51, 51, 51, 38, 13, 51, 39, 13, 15, 27, 33, 36, 4, 51, 51, 51, 51, 51, 51, 51, 51, 51, 33, 6, 51, 51, 51, 51, 24, 6, 51, 51, 51, 7, 51, 7, 24, 9, 51, 51, 21, 51, 1, 19, 51, 9, 25, 51, 51, 22, 1, 51, 23, 1, 51, 22, 1, 51, 23, 1, 51, 51, 1, 19, 1, 14, 51, 2, 51, 51, 51, 21, 31, 51, 51, 51, 51, 51, 51, 51, 51, 51, 35, 37, 51, 51, 51, 51, 51, 51, 51, 51, 51, 20, 50, 51, 51, 5, 51, 46, 28, 51, 26, 2, 8, 47, 5, 17, 32, 3, 10, 12, 2, 5, 17, 51, 51, 45, 2, 16, 3, 51, 49, 51, 4, 16, 3, 18, 10, 12, 40, 6, 14, 8, 2, 48, 4, 15, 8, 4, 3, 7, 51, 20, 51, 43, 51, 41, 6, 51, 51, 51, 51, 51, 51, 51, 51, 51, 38, 13, 39, 13, 4, 15, 27, 33, 36, 51, 34, 51, 51, 51, 51, 5, 51, 51, 51, 6, 51,

지금까지 파이썬의 dictionary 자료형으로 정수 인코딩을 진행해보았습니다. 그런데 이보다는 좀 더 쉽게 하기 위해서 Counter, FreqDist, enumerate 또는 케라스 토크나이저를 사용하는 것을 권장합니다.

### 2) Counter 사용하기

In [0]:
from collections import Counter

In [37]:
print(sentences)

[['올리브', '이정', '부대', '경기도', '의정부시', '평화', '552', '031', '856', '5290', '1274446611', '유진', 'tel', '유진', '2020', '-02', '-24', '18', '40', '33', '1001', '00077', '고객', '센터', '1577', '4887', 'oliveyoung', '결제', '일로', '부터', '15', '10', '판매', '가능', '상품', '개봉', '결제', '수단', '구매', '영수증', '매장', '방문', '가능', '결제', '변경', '매매장', '에서', '가능', '부분', '영수증', '의하', '불은', '불가', '부분', '영수증', '선불', '매장', '방문', '해당', '금액', '이상', '상품', '기능', '증정', '불가', '클리어런스', '상품', '불가', '발행', '영수증', '카드', '전표', '고객', '상품명', '단가', '수량', '금액', '10', '쏘피', '오가', '크트', '라이더', '일반', '매가', '행사', '8809299236129', '400', '400', '표시', '품목', '가세', '면세', '품목', '클리어런스', '쿠폰', '제휴', '중복', '불가', 'oli', '944', '||', '4,860', '판매계', '400', '신용카드', '14', '860', '제휴', '금액', '540', '계제', '4,660', '세대', '승인', '금액', '4,860', '5433', '334', '카드', '현대', '카드', '승인', '번호', '00146478', '일시불', 'cj', 'one', '포인트', '내역', '회원', '번호', '**', '***', '023635', '적립', '포인트', '24', '가용', '포인트', '15,319', '적립', '포인트', '익일', '가용', '포인트', '전환', '자세', '포인트', '내

현재 sentences는 단어 토큰화가 된 결과가 저장되어져 있습니다. 단어 집합(vocabulary)을 만들기 위해서 sentences에서 문장의 경계인 [, ]를 제거하고 단어들을 하나의 리스트로 만들겠습니다.

In [38]:
words=sum(sentences, [])
# 위 작업은 words = np.hstack(sentences)로도 수행 가능.
print(words)

['올리브', '이정', '부대', '경기도', '의정부시', '평화', '552', '031', '856', '5290', '1274446611', '유진', 'tel', '유진', '2020', '-02', '-24', '18', '40', '33', '1001', '00077', '고객', '센터', '1577', '4887', 'oliveyoung', '결제', '일로', '부터', '15', '10', '판매', '가능', '상품', '개봉', '결제', '수단', '구매', '영수증', '매장', '방문', '가능', '결제', '변경', '매매장', '에서', '가능', '부분', '영수증', '의하', '불은', '불가', '부분', '영수증', '선불', '매장', '방문', '해당', '금액', '이상', '상품', '기능', '증정', '불가', '클리어런스', '상품', '불가', '발행', '영수증', '카드', '전표', '고객', '상품명', '단가', '수량', '금액', '10', '쏘피', '오가', '크트', '라이더', '일반', '매가', '행사', '8809299236129', '400', '400', '표시', '품목', '가세', '면세', '품목', '클리어런스', '쿠폰', '제휴', '중복', '불가', 'oli', '944', '||', '4,860', '판매계', '400', '신용카드', '14', '860', '제휴', '금액', '540', '계제', '4,660', '세대', '승인', '금액', '4,860', '5433', '334', '카드', '현대', '카드', '승인', '번호', '00146478', '일시불', 'cj', 'one', '포인트', '내역', '회원', '번호', '**', '***', '023635', '적립', '포인트', '24', '가용', '포인트', '15,319', '적립', '포인트', '익일', '가용', '포인트', '전환', '자세', '포인트', '내역

이를 파이썬의 Counter()의 입력으로 사용하면 중복을 제거하고 단어의 빈도수를 기록합니다.

In [39]:
vocab=Counter(words) # 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산할 수 있습니다.
print(vocab)

Counter({'포인트': 1256, '가능': 886, '영수증': 762, '불가': 737, '결제': 715, '금액': 689, '카드': 618, '상품': 611, '번호': 471, '매장': 469, '000': 416, '방문': 402, '품목': 397, '이상': 387, '클리어런스': 386, '부분': 380, '수단': 372, '선물': 371, '내역': 364, '고객': 358, 'cj': 357, '적립': 355, '가용': 355, '승인': 344, '**': 317, '판매': 314, '쿠폰': 296, '15': 277, '행사': 241, '매가': 233, '올리브': 232, '구매': 223, '제휴': 219, '합계': 216, 'tel': 210, '중복': 207, '02': 206, '표시': 203, '면세': 202, '해당': 201, '수량': 200, '변경': 199, '상품명': 197, '단가': 197, '에서': 195, '부터': 194, '개봉': 194, '증정': 194, '의하': 192, '센터': 189, '10': 189, '과세': 187, 'one': 185, '1577': 182, '사용': 182, '익일': 181, '일로': 178, '자세': 177, '일시불': 171, 'oliveyoung': 165, '2020': 164, '전환': 164, '매매장': 162, '12': 162, '불은': 160, '전표': 160, '회원': 159, '900': 157, '11': 151, '신용카드': 148, '4887': 140, '500': 134, '8098101574': 125, '18': 111, '100': 105, '구창': 103, '13': 88, '800': 86, '1001': 84, '19': 82, '****': 79, '20': 77, '1002': 77, '26': 77, '400': 76, '14': 75, '부가세': 

most_common()는 상위 빈도수를 가진 주어진 수의 단어만을 리턴합니다. 이를 사용하여 등장 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있습니다. 등장 빈도수 상위 5개의 단어만 단어 집합으로 저장해봅시다.

In [40]:
vocab_size=50
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 50개의 단어만 저장
vocab

[('포인트', 1256),
 ('가능', 886),
 ('영수증', 762),
 ('불가', 737),
 ('결제', 715),
 ('금액', 689),
 ('카드', 618),
 ('상품', 611),
 ('번호', 471),
 ('매장', 469),
 ('000', 416),
 ('방문', 402),
 ('품목', 397),
 ('이상', 387),
 ('클리어런스', 386),
 ('부분', 380),
 ('수단', 372),
 ('선물', 371),
 ('내역', 364),
 ('고객', 358),
 ('cj', 357),
 ('적립', 355),
 ('가용', 355),
 ('승인', 344),
 ('**', 317),
 ('판매', 314),
 ('쿠폰', 296),
 ('15', 277),
 ('행사', 241),
 ('매가', 233),
 ('올리브', 232),
 ('구매', 223),
 ('제휴', 219),
 ('합계', 216),
 ('tel', 210),
 ('중복', 207),
 ('02', 206),
 ('표시', 203),
 ('면세', 202),
 ('해당', 201),
 ('수량', 200),
 ('변경', 199),
 ('상품명', 197),
 ('단가', 197),
 ('에서', 195),
 ('부터', 194),
 ('개봉', 194),
 ('증정', 194),
 ('의하', 192),
 ('센터', 189)]

이제 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여합니다.

In [41]:
word_to_index={}
i=0
for (word, frequency) in vocab :
    i=i+1
    word_to_index[word]=i
print(word_to_index)

{'포인트': 1, '가능': 2, '영수증': 3, '불가': 4, '결제': 5, '금액': 6, '카드': 7, '상품': 8, '번호': 9, '매장': 10, '000': 11, '방문': 12, '품목': 13, '이상': 14, '클리어런스': 15, '부분': 16, '수단': 17, '선물': 18, '내역': 19, '고객': 20, 'cj': 21, '적립': 22, '가용': 23, '승인': 24, '**': 25, '판매': 26, '쿠폰': 27, '15': 28, '행사': 29, '매가': 30, '올리브': 31, '구매': 32, '제휴': 33, '합계': 34, 'tel': 35, '중복': 36, '02': 37, '표시': 38, '면세': 39, '해당': 40, '수량': 41, '변경': 42, '상품명': 43, '단가': 44, '에서': 45, '부터': 46, '개봉': 47, '증정': 48, '의하': 49, '센터': 50}


## 케라스(Keras)의 텍스트 전처리

케라스(Keras)는 기본적인 전처리를 위한 도구들을 제공합니다. 때로는 정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저를 사용하기도 하는데, 사용 방법과 그 특징에 대해서 이해해보겠습니다.

In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.

fit_on_texts는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여하는데, 정확히 앞서 설명한 정수 인코딩 작업이 이루어진다고 보면됩니다. 각 단어에 인덱스가 어떻게 부여되었는지를 보려면, word_index를 사용합니다.

In [44]:
print(tokenizer.word_index)

{'포인트': 1, '가능': 2, '영수증': 3, '불가': 4, '결제': 5, '금액': 6, '카드': 7, '상품': 8, '번호': 9, '매장': 10, '000': 11, '방문': 12, '품목': 13, '이상': 14, '클리어런스': 15, '부분': 16, '수단': 17, '선물': 18, '내역': 19, '고객': 20, 'cj': 21, '적립': 22, '가용': 23, '승인': 24, '**': 25, '판매': 26, '쿠폰': 27, '15': 28, '행사': 29, '매가': 30, '올리브': 31, '구매': 32, '제휴': 33, '합계': 34, 'tel': 35, '중복': 36, '02': 37, '표시': 38, '면세': 39, '해당': 40, '수량': 41, '변경': 42, '상품명': 43, '단가': 44, '에서': 45, '부터': 46, '개봉': 47, '증정': 48, '의하': 49, '센터': 50, '10': 51, '과세': 52, 'one': 53, '1577': 54, '사용': 55, '익일': 56, '일로': 57, '자세': 58, '일시불': 59, 'oliveyoung': 60, '2020': 61, '전환': 62, '매매장': 63, '12': 64, '불은': 65, '전표': 66, '회원': 67, '900': 68, '11': 69, '신용카드': 70, '4887': 71, '500': 72, '8098101574': 73, '18': 74, '100': 75, '구창': 76, '13': 77, '800': 78, '1001': 79, '19': 80, '****': 81, '20': 82, '1002': 83, '26': 84, '400': 85, '14': 86, '부가세': 87, '200': 88, 'young': 89, 'olive': 90, '사명': 91, '서울특별시': 92, '***': 93, '300': 94, '16': 95

각 단어의 빈도수가 높은 순서대로 인덱스가 부여된 것을 확인할 수 있습니다. 각 단어가 카운트를 수행하였을 때 몇 개였는지를 보고자 한다면 word_counts를 사용합니다.

In [45]:
print(tokenizer.word_counts)

OrderedDict([('올리브', 232), ('이정', 2), ('부대', 1), ('경기도', 35), ('의정부시', 2), ('평화', 1), ('552', 1), ('031', 33), ('856', 1), ('5290', 40), ('1274446611', 1), ('유진', 4), ('tel', 210), ('2020', 164), ('-02', 44), ('-24', 4), ('18', 111), ('40', 21), ('33', 12), ('1001', 84), ('00077', 1), ('고객', 358), ('센터', 189), ('1577', 182), ('4887', 140), ('oliveyoung', 165), ('결제', 715), ('일로', 178), ('부터', 194), ('15', 277), ('10', 189), ('판매', 314), ('가능', 886), ('상품', 611), ('개봉', 194), ('수단', 372), ('구매', 223), ('영수증', 762), ('매장', 469), ('방문', 402), ('변경', 199), ('매매장', 162), ('에서', 195), ('부분', 380), ('의하', 192), ('불은', 160), ('불가', 737), ('선불', 2), ('해당', 201), ('금액', 689), ('이상', 387), ('기능', 26), ('증정', 194), ('클리어런스', 386), ('발행', 38), ('카드', 618), ('전표', 160), ('상품명', 197), ('단가', 197), ('수량', 200), ('쏘피', 4), ('오가', 1), ('크트', 1), ('라이더', 1), ('일반', 1), ('매가', 233), ('행사', 241), ('8809299236129', 1), ('400', 76), ('표시', 203), ('품목', 397), ('가세', 15), ('면세', 202), ('쿠폰', 296), ('제휴', 219),

texts_to_sequences()는 입력으로 들어온 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환합니다.

In [46]:
print(tokenizer.texts_to_sequences(sentences))

[[31, 901, 1528, 115, 902, 1529, 1530, 117, 1531, 109, 1532, 488, 35, 488, 61, 106, 489, 74, 142, 201, 79, 1533, 20, 50, 54, 71, 60, 5, 57, 46, 28, 51, 26, 2, 8, 47, 5, 17, 32, 3, 10, 12, 2, 5, 42, 63, 45, 2, 16, 3, 49, 65, 4, 16, 3, 903, 10, 12, 40, 6, 14, 8, 126, 48, 4, 15, 8, 4, 111, 3, 7, 66, 20, 43, 44, 41, 6, 51, 490, 1534, 1535, 1536, 1537, 30, 29, 1538, 85, 85, 38, 13, 169, 39, 13, 15, 27, 33, 36, 4, 236, 1539, 357, 904, 112, 85, 70, 86, 1540, 33, 6, 905, 641, 1541, 1542, 24, 6, 904, 1543, 1544, 7, 170, 7, 24, 9, 1545, 59, 21, 53, 1, 19, 67, 9, 25, 93, 1546, 22, 1, 118, 23, 1, 1547, 22, 1, 56, 23, 1, 62, 58, 1, 19, 1, 14, 55, 2, 90, 89, 1548, 21, 31, 121, 92, 312, 491, 413, 642, 202, 492, 1549, 35, 37, 414, 127, 140, 313, 97, 237, 86, 1550, 643, 20, 50, 187, 60, 5, 57, 46, 28, 82, 26, 2, 8, 47, 5, 17, 32, 3, 10, 12, 2, 5, 17, 1551, 63, 45, 2, 16, 3, 177, 49, 65, 4, 16, 3, 18, 10, 12, 40, 6, 14, 8, 2, 48, 4, 15, 8, 4, 3, 7, 66, 20, 906, 43, 358, 41, 6, 75, 217, 283, 314, 218, 72

앞서 빈도수가 가장 높은 단어 n개만을 사용하기 위해서 most_common()을 사용했었습니다. 케라스 토크나이저에서는 tokenizer = Tokenizer(num_words=숫자)와 같은 방법으로 빈도수가 높은 상위 몇 개의 단어만 사용하겠다고 지정할 수 있습니다. 여기서는 1번 단어부터 100번 단어까지만 사용하겠습니다. 상위 5개 단어를 사용한다고 토크나이저를 재정의 해보겠습니다.

In [0]:
vocab_size=10
tokenizer = Tokenizer(num_words=vocab_size+1) # 상위 5개 단어만 사용
tokenizer.fit_on_texts(sentences)

num_words에서 +1을 더해서 값을 넣어주는 이유는 num_words는 숫자를 0부터 카운트합니다. 만약 5를 넣으면 0 ~ 4번 단어 보존을 의미하게 되므로 뒤의 실습에서 1번 단어부터 4번 단어만 남게됩니다. 그렇기 때문에 1 ~ 5번 단어까지 사용하고 싶다면 num_words에 숫자 5를 넣어주는 것이 아니라 5+1인 값을 넣어주어야 합니다.

실질적으로 숫자 0에 지정된 단어가 존재하지 않는데도 케라스 토크나이저가 숫자 0까지 단어 집합의 크기로 산정하는 이유는 자연어 처리에서 패딩(padding)이라는 작업 때문입니다. 이에 대해서는 뒤에 다루게 되므로 여기서는 케라스 토크나이저를 사용할 때는 숫자 0도 단어 집합의 크기로 고려해야한다고만 이해합시다.

다시 word_index를 확인해보겠습니다.

In [54]:
print(tokenizer.word_index)

{'포인트': 1, '가능': 2, '영수증': 3, '불가': 4, '결제': 5, '금액': 6, '카드': 7, '상품': 8, '번호': 9, '매장': 10, '000': 11, '방문': 12, '품목': 13, '이상': 14, '클리어런스': 15, '부분': 16, '수단': 17, '선물': 18, '내역': 19, '고객': 20, 'cj': 21, '적립': 22, '가용': 23, '승인': 24, '**': 25, '판매': 26, '쿠폰': 27, '15': 28, '행사': 29, '매가': 30, '올리브': 31, '구매': 32, '제휴': 33, '합계': 34, 'tel': 35, '중복': 36, '02': 37, '표시': 38, '면세': 39, '해당': 40, '수량': 41, '변경': 42, '상품명': 43, '단가': 44, '에서': 45, '부터': 46, '개봉': 47, '증정': 48, '의하': 49, '센터': 50, '10': 51, '과세': 52, 'one': 53, '1577': 54, '사용': 55, '익일': 56, '일로': 57, '자세': 58, '일시불': 59, 'oliveyoung': 60, '2020': 61, '전환': 62, '매매장': 63, '12': 64, '불은': 65, '전표': 66, '회원': 67, '900': 68, '11': 69, '신용카드': 70, '4887': 71, '500': 72, '8098101574': 73, '18': 74, '100': 75, '구창': 76, '13': 77, '800': 78, '1001': 79, '19': 80, '****': 81, '20': 82, '1002': 83, '26': 84, '400': 85, '14': 86, '부가세': 87, '200': 88, 'young': 89, 'olive': 90, '사명': 91, '서울특별시': 92, '***': 93, '300': 94, '16': 95

사실 실제 적용은 texts_to_sequences를 사용할 때 적용이 됩니다.

In [55]:
print(tokenizer.texts_to_sequences(sentences))

[[5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 4, 8, 4, 3, 7, 6, 4, 6, 6, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 3, 7, 6, 4, 5, 6, 9, 7, 7, 1, 9, 1, 1, 1, 1, 1, 1, 2, 2, 8, 5, 3, 2, 5, 3, 3, 10, 6, 8, 2, 4, 8, 4, 3, 7, 8, 4, 7, 1, 1, 9, 5, 5, 2, 8, 2, 3, 10, 10, 5, 2, 3, 4, 3, 8, 10, 6, 2, 4, 8, 4, 3, 7, 6, 4, 7, 5, 6, 7, 9, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 3, 7, 6, 4, 7, 6, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 10, 3, 3, 10, 10, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 7, 6, 3, 10, 7, 6, 4, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 3, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 5, 5, 2, 8, 3, 10, 5, 3, 4, 3, 8, 10, 6, 2, 4, 4, 8, 3, 7, 6, 4, 7, 6, 7, 9, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 3, 10, 10, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 7, 6, 3, 10, 4, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 2, 8, 5, 3, 

케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있습니다. 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용합니다.

In [0]:
tokenizer = Tokenizer() # num_words를 여기서는 지정하지 않은 상태
tokenizer.fit_on_texts(sentences)

In [59]:
vocab_size=20
words_frequency = [w for w,c in tokenizer.word_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del tokenizer.word_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[w] # 해당 단어에 대한 카운트 정보를 삭제
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'포인트': 1, '가능': 2, '영수증': 3, '불가': 4, '결제': 5, '금액': 6, '카드': 7, '상품': 8, '번호': 9, '매장': 10}
OrderedDict([('결제', 715), ('가능', 886), ('상품', 611), ('영수증', 762), ('매장', 469), ('불가', 737), ('금액', 689), ('카드', 618), ('번호', 471), ('포인트', 1256)])
[[5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 4, 8, 4, 3, 7, 6, 4, 6, 6, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 3, 7, 6, 4, 5, 6, 9, 7, 7, 1, 9, 1, 1, 1, 1, 1, 1, 2, 2, 8, 5, 3, 2, 5, 3, 3, 10, 6, 8, 2, 4, 8, 4, 3, 7, 8, 4, 7, 1, 1, 9, 5, 5, 2, 8, 2, 3, 10, 10, 5, 2, 3, 4, 3, 8, 10, 6, 2, 4, 8, 4, 3, 7, 6, 4, 7, 5, 6, 7, 9, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 3, 7, 6, 4, 7, 6, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 10, 3, 3, 10, 10, 5, 2, 8, 5, 3, 10, 2, 5, 2, 3, 4, 3, 10, 6, 8, 2, 4, 8, 4, 7, 6, 3, 10, 7, 6, 4, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 3, 5, 6, 6, 7, 9, 7, 7, 9, 1, 9, 1, 1, 1, 1, 1, 1, 2, 5, 5, 5, 2, 8, 3, 10, 5, 3,

In [0]:
vocab_size=5
tokenizer = Tokenizer(num_words=vocab_size+2, oov_token='OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)

만약 oov_token을 사용하기로 했다면 케라스 토크나이저는 기본적으로 'OOV'의 인덱스를 1로 합니다.

In [61]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


이제 코퍼스에 대해서 정수 인코딩을 진행합니다.

In [62]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 3, 1, 1, 6, 1, 1, 4, 1, 1, 3, 6, 1, 1, 1, 3, 1, 4, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 3, 1, 1, 6, 1, 1, 4, 1, 1, 3, 6, 1, 1, 1, 1, 3, 1, 4, 1, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 1, 1, 3, 1, 5, 1, 1, 5, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

빈도수 상위 10개의 단어는 2 ~ 10까지의 인덱스를 가졌으며, 그 외 단어 집합에 없는 'good'과 같은 단어들은 전부 'OOV'의 인덱스인 1로 인코딩되었습니다.